# **Import libraries**

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests as rq
import datetime
import requests

# **Creating global variables**

In [ ]:
global rating, Aircraft, Class, Travel_type, Route, Date, comfort, staff, food, entertainment, wifi, ground_service, vfm, recommend

verified=[]
rating=[]
feedback=[]
Aircraft=[]
Class=[]
Travel_type=[]
Route=[]
Date=[]
comfort=[]
staff=[]
food=[]
entertainment=[]
wifi=[]
ground_service=[]
vfm=[]
recommend=[]


# **Extracting specific attributes and ratings from the HTML table rows**

In [ ]:
def value_fetcher(soup):
    rating.append(np.nan)
    Aircraft.append(np.nan)
    Class.append(np.nan)
    Travel_type.append(np.nan)
    Route.append(np.nan)
    Date.append(np.nan)
    comfort.append(np.nan)
    staff.append(np.nan)
    food.append(np.nan)
    entertainment.append(np.nan)
    wifi.append(np.nan)
    ground_service.append(np.nan)
    vfm.append(np.nan)
    recommend.append(np.nan)

    for row in soup.find_all('tr'):
        header=row.find('td',class_='review-rating-header').text
        value=row.find('td',class_='review-value')
        if header=='Aircraft':
            Aircraft[-1]=value.text
        elif header=='Type Of Traveller':
            Travel_type[-1]=value.text
        elif header=='Seat Type':
            Class[-1]=value.text
        elif header=='Route':
            Route[-1]=value.text
        elif header=='Date Flown':
            Date[-1]=value.text
        elif header=='Seat Comfort':
            comfort[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Cabin Staff Service':
            staff[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Food & Beverages':
            food[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Inflight Entertainment':
            entertainment[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Ground Service':
            ground_service[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Value For Money':
            vfm[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Recommended':
            recommend[-1]=value.text
        elif header=='Wifi & Connectivity':
            wifi[-1]=len(row.find_all('span',class_='star fill'))

In [ ]:
def fetcher(element, tag, attribute, value):
    result = element.find(tag, {attribute: value})
    return result.text if result else None

In [ ]:
pages = 1000
page_size = 1000
review_limit = 5624  # Desired number of reviews

base_url = "https://www.airlinequality.com/airline-reviews/american-airlines"

review_count = 0  # Initialize review count

# Iterate over the pages until the desired number of reviews is reached
for page in range(1, pages + 1):
    url = f'{base_url}/page/{page}/?sortby=post_date%3ADesc&pagesize={page_size}'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, 'lxml')
    articles = soup.find_all('article', {'itemprop': 'review'})
    for article in articles:
        rating.append(fetcher(article, 'span', 'itemprop', 'ratingValue'))
        div = article.find('div', class_='tc_mobile')
        content = (div.find('div', class_='text_content').text.split('|'))
        if len(content) == 2:
            verification, review = content
        else:
            verification = np.nan
            review = content[0]
        verified.append(verification)
        feedback.append(review)
        table = div.find('table', class_='review-ratings')
        value_fetcher(table)

        review_count += 1  # Increment review count

        if review_count >= review_limit:
            break  # Exit the loop if the desired number of reviews is reached

    if review_count >= review_limit:
        break  # Exit the loop if the desired number of reviews is reached

    print(f"   ---> {review_count} reviews scraped")

print(f"Total reviews scraped: {review_count}")


   ---> 1000 reviews scraped
   ---> 2000 reviews scraped
   ---> 3000 reviews scraped
   ---> 4000 reviews scraped
   ---> 5000 reviews scraped
Total reviews scraped: 5624


In [ ]:
data.to_csv('american(20000).csv',index=False)